In [1]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark
!!pip install pyspark

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


['Requirement already satisfied: pyspark in /usr/local/lib/python3.10/dist-packages (3.5.2)',
 'Requirement already satisfied: py4j==0.10.9.7 in /usr/local/lib/python3.10/dist-packages (from pyspark) (0.10.9.7)']

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [4]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
get_ipython().system_raw('./ngrok http 4050 &')

In [6]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType
from pyspark.sql.functions import col
from pyspark.sql import functions as f
import os
import pandas as pd

In [4]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Lendo CSVs") \
    .getOrCreate()

In [5]:
# DEFININDO PATHS DOS ARQUIVOS

empresas = '/content/drive/MyDrive/Colab Notebooks/data_bases/empresas'
estabelecimentos = '/content/drive/MyDrive/Colab Notebooks/data_bases/estabelecimentos'
socios = '/content/drive/MyDrive/Colab Notebooks/data_bases/socios'

In [6]:
# Lendo CSVs
empresas = spark.read.csv(empresas, sep=';', inferSchema=True)
estabelecimentos = spark.read.csv(estabelecimentos, sep=';', inferSchema=True)
socios = spark.read.csv(socios, sep=';', inferSchema=True)

In [7]:
# RENOMENADO AS COLUNAS

In [8]:
empresas.limit(5).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None


In [9]:
empresasColNames = ['cnpj_basico', 'razao_social', 'natureza_juridica', 'qualificação_resposavel', 'capital_social_empresa', 'porte_da_empresa', 'ente-federativo_responsavel']

In [10]:
for index, colName in enumerate(empresasColNames):
    empresas = empresas.withColumnRenamed(f"_c{index}", colName)

empresas.columns

['cnpj_basico',
 'razao_social',
 'natureza_juridica',
 'qualificação_resposavel',
 'capital_social_empresa',
 'porte_da_empresa',
 'ente-federativo_responsavel']

In [11]:
# CONVERTENDO STRING EM DOUBLE NA TABELA EMPRESAS

In [12]:
empresas = empresas.withColumn('capital_social_empresa', f.regexp_replace('capital_social_empresa', ',', '.'))
empresas.limit(5).toPandas()

,cnpj_basico,razao_social,natureza_juridica,qualificação_resposavel,capital_social_empresa,porte_da_empresa,ente-federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.00,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.00,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.00,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.00,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.00,1,None


In [13]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificação_resposavel: integer (nullable = true)
 |-- capital_social_empresa: string (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente-federativo_responsavel: string (nullable = true)



In [14]:
empresas = empresas.withColumn('capital_social_empresa', empresas['capital_social_empresa'].cast(DoubleType()))
empresas.limit(5).toPandas()

,cnpj_basico,razao_social,natureza_juridica,qualificação_resposavel,capital_social_empresa,porte_da_empresa,ente-federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.0,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.0,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.0,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.0,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.0,1,None


In [15]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificação_resposavel: integer (nullable = true)
 |-- capital_social_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente-federativo_responsavel: string (nullable = true)



In [16]:
estabelecimentosColNames = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']

In [17]:
for index, colName in enumerate(estabelecimentosColNames):
    estabelecimentos = estabelecimentos.withColumnRenamed(f"_c{index}", colName)

estabelecimentos.columns

['cnpj_basico',
 'cnpj_ordem',
 'cnpj_dv',
 'identificador_matriz_filial',
 'nome_fantasia',
 'situacao_cadastral',
 'data_situacao_cadastral',
 'motivo_situacao_cadastral',
 'nome_da_cidade_no_exterior',
 'pais',
 'data_de_inicio_atividade',
 'cnae_fiscal_principal',
 'cnae_fiscal_secundaria',
 'tipo_de_logradouro',
 'logradouro',
 'numero',
 'complemento',
 'bairro',
 'cep',
 'uf',
 'municipio',
 'ddd_1',
 'telefone_1',
 'ddd_2',
 'telefone_2',
 'ddd_do_fax',
 'fax',
 'correio_eletronico',
 'situacao_especial',
 'data_da_situacao_especial']

In [18]:
sociosColNames = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

In [19]:
for index, colName in enumerate(sociosColNames):
    socios = socios.withColumnRenamed(f"_c{index}", colName)

socios.columns

['cnpj_basico',
 'identificador_de_socio',
 'nome_do_socio_ou_razao_social',
 'cnpj_ou_cpf_do_socio',
 'qualificacao_do_socio',
 'data_de_entrada_sociedade',
 'pais',
 'representante_legal',
 'nome_do_representante',
 'qualificacao_do_representante_legal',
 'faixa_etaria']

In [20]:
empresas.limit(2).toPandas()

,cnpj_basico,razao_social,natureza_juridica,qualificação_resposavel,capital_social_empresa,porte_da_empresa,ente-federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.0,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.0,5,None


In [21]:
estabelecimentos.limit(2).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dv,identificador_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_da_cidade_no_exterior,pais,...,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_do_fax,fax,correio_eletronico,situacao_especial,data_da_situacao_especial
0,4519,1,48,1,GIRAFFAS,8,19950331,1,None,None,...,6219,None,None,None,None,NaN,None,None,None,NaN
1,8638,1,79,1,AGROPECUARIA FAGUNDES,8,20150209,73,None,None,...,7255,None,None,None,None,NaN,None,None,None,NaN


In [22]:
socios.limit(2).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,19940725,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,19940725,NaN,***000000**,None,0,7


In [23]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificação_resposavel: integer (nullable = true)
 |-- capital_social_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente-federativo_responsavel: string (nullable = true)



In [24]:
socios.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: integer (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [25]:
estabelecimentos.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: string (nullable = true)
 |-- data_situacao_cadastral: string (nullable = true)
 |-- motivo_situacao_cadastral: string (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- data_de_inicio_atividade: integer (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: string 

In [26]:
# Mudando valor das colunas datas de string/integer para valor date

In [27]:
# Alterando no DF estabelecimentos
estabelecimentos = estabelecimentos\
    .withColumn(
        "data_situacao_cadastral",
        f.to_date(estabelecimentos.data_situacao_cadastral.cast(StringType()), "yyyyMMdd")
    )\
    .withColumn(
        "data_de_inicio_atividade",
        f.to_date(estabelecimentos.data_de_inicio_atividade.cast(StringType()), "yyyyMMdd")
    )\
    .withColumn(
        "data_da_situacao_especial",
        f.to_date(estabelecimentos.data_da_situacao_especial.cast(StringType()), "yyyyMMdd")
    )


In [28]:
estabelecimentos.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: string (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: string (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- data_de_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: string (null

In [29]:
# Alterando no DF socios
socios = socios\
    .withColumn(
        "data_de_entrada_sociedade",
        f.to_date(socios.data_de_entrada_sociedade.cast(StringType()), "yyyyMMdd"))

In [30]:
socios.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: date (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [31]:
# Realizando consultas com os dfs e criando novos dfs

In [32]:
empresas\
    .select('natureza_juridica', 'porte_da_empresa', 'capital_social_empresa')\
    .show(5)

+-----------------+----------------+----------------------+
|natureza_juridica|porte_da_empresa|capital_social_empresa|
+-----------------+----------------+----------------------+
|             2240|               1|                   0.0|
|             2062|               5|                   0.0|
|             3034|               5|                   0.0|
|             2135|               5|                   0.0|
|             2062|               1|                4000.0|
+-----------------+----------------+----------------------+
only showing top 5 rows



In [33]:
socios\
    .select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .show(5)

+-----------------------------+------------+--------------+
|nome_do_socio_ou_razao_social|faixa_etaria|ano_de_entrada|
+-----------------------------+------------+--------------+
|         LILIANA PATRICIA ...|           7|          1994|
|         CRISTINA HUNDERTMARK|           7|          1994|
|         CELSO EDUARDO DE ...|           8|          1994|
|         EDUARDO BERRINGER...|           5|          1994|
|          HANNE MAHFOUD FADEL|           8|          1994|
+-----------------------------+------------+--------------+
only showing top 5 rows



In [34]:
estabelecimentos\
    .select('nome_fantasia', 'municipio', f.year('data_de_inicio_atividade').alias('ano_de_inicio_atividade '),
            f.month('data_de_inicio_atividade').alias('mes_de_inicio_atividade'))\
    .show(5)

+--------------------+---------+------------------------+-----------------------+
|       nome_fantasia|municipio|ano_de_inicio_atividade |mes_de_inicio_atividade|
+--------------------+---------+------------------------+-----------------------+
|            GIRAFFAS|     6219|                    1994|                      5|
|AGROPECUARIA FAGU...|     7255|                    1994|                      5|
|                NULL|     7097|                    1994|                      5|
|                NULL|     7107|                    1994|                      6|
|JS MATERIAIS DE C...|     6915|                    1994|                      6|
+--------------------+---------+------------------------+-----------------------+
only showing top 5 rows



In [35]:
# Identificando valores nulos

In [36]:
socios.limit(2).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,1994-07-25,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,1994-07-25,NaN,***000000**,None,0,7


In [37]:
socios.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in socios.columns]).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,0,0,208,1234,0,1,2038255,0,1995432,0,0


In [38]:
socios.na.fill(0).limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,1994-07-25,0,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,1994-07-25,0,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,1994-05-16,0,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,1994-05-16,0,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,1994-06-09,0,***000000**,None,0,8


In [39]:
socios.na.fill('-').limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,1994-07-25,NaN,***000000**,-,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,1994-07-25,NaN,***000000**,-,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,1994-05-16,NaN,***000000**,-,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,1994-05-16,NaN,***000000**,-,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,1994-06-09,NaN,***000000**,-,0,8


In [40]:
# ORDENANDO OS DADOS

In [41]:
socios\
    .select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .orderBy('ano_de_entrada', ascending=False)\
    .show(5, False)

+----------------------------------------+------------+--------------+
|nome_do_socio_ou_razao_social           |faixa_etaria|ano_de_entrada|
+----------------------------------------+------------+--------------+
|KASSIANO RODRIGO KICHILESKI             |4           |2021          |
|JOSE HUMBERTO PAIVA                     |6           |2021          |
|LEONARDO MENNA BARRETO LARANJA GONCALVES|5           |2021          |
|BENILDES BARBOSA RODRIGUES              |8           |2021          |
|MANOEL ADRIANO COSTA BARBOSA            |6           |2021          |
+----------------------------------------+------------+--------------+
only showing top 5 rows



In [42]:
socios\
    .select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .orderBy(['ano_de_entrada', 'faixa_etaria'], ascending=[False, False])\
    .show(10, False)

+-------------------------------------+------------+--------------+
|nome_do_socio_ou_razao_social        |faixa_etaria|ano_de_entrada|
+-------------------------------------+------------+--------------+
|MARIA RAIMUNDA DOS SANTOS LANZA      |9           |2021          |
|ANTONIO TAVARES DE ANDRADE           |9           |2021          |
|RENILDE DAS GRACAS MAIA              |9           |2021          |
|ANNA MARIA TELLES FERREIRA SANTOS    |9           |2021          |
|DORIS PEREIRA GOMES JAZRA            |9           |2021          |
|ANTONIA DE SOUSA VIEIRA              |9           |2021          |
|MARIA JOSE DOMINGUES BONATO          |9           |2021          |
|AURA MARIA DE ANDRADE                |9           |2021          |
|ZELIA MARIA CAMARA RODRIGUES DA SILVA|9           |2021          |
|SONIA MARQUES SAMAJA                 |9           |2021          |
+-------------------------------------+------------+--------------+
only showing top 10 rows



In [43]:
# FILTRANDO OS DADOS
# Utilizando, Filter, Where, Select and Like

In [44]:
empresas\
    .where('capital_social_empresa==50')\
    .show(5, False)

+-----------+------------------------------------+-----------------+-----------------------+----------------------+----------------+---------------------------+
|cnpj_basico|razao_social                        |natureza_juridica|qualificação_resposavel|capital_social_empresa|porte_da_empresa|ente-federativo_responsavel|
+-----------+------------------------------------+-----------------+-----------------------+----------------------+----------------+---------------------------+
|17350147   |ERIK MARCELO DOS SANTOS 42107848858 |2135             |50                     |50.0                  |1               |NULL                       |
|17833214   |ALEXANDRE MACHADO LIMA 73750123772  |2135             |50                     |50.0                  |1               |NULL                       |
|20860830   |YASMIN MOURA DA FONSECA 13457709793 |2135             |50                     |50.0                  |1               |NULL                       |
|22242856   |JOAO CESAR MESSIAS 08

In [45]:
socios\
    .select('nome_do_socio_ou_razao_social')\
    .filter(socios.nome_do_socio_ou_razao_social.startswith("MARCOS"))\
    .filter(socios.nome_do_socio_ou_razao_social.endswith("PEREIRA"))\
    .limit(10)\
    .toPandas()

,nome_do_socio_ou_razao_social
0,MARCOS ANTONIO LOPES PEREIRA
1,MARCOS WILSON PEREIRA
2,MARCOS PAULO BERNARDO PEREIRA
3,MARCOS GEOVANNE BARRETO PEREIRA
4,MARCOS ANTONIO VARGAS PEREIRA
5,MARCOS EDUARDO PEREIRA
6,MARCOS ANTONIO BARROZO PEREIRA
7,MARCOS ROGERIO RODRIGUES PEREIRA
8,MARCOS ANTONIO PEREIRA
9,MARCOS DE OLIVEIRA PEREIRA


In [46]:
# EXERCICIO

In [47]:
# data = [
#     ('CARMINA RABELO', 4, 2010),
#     ('HERONDINA PEREIRA', 6, 2009),
#     ('IRANI DOS SANTOS', 12, 2010),
#     ('JOAO BOSCO DA FONSECA', 3, 2009),
#     ('CARLITO SOUZA', 1, 2010),
#     ('WALTER DIAS', 9, 2009),
#     ('BRENO VENTUROSO', 1, 2009),
#     ('ADELINA TEIXEIRA', 5, 2009),
#     ('ELIO SILVA', 7, 2010),
#     ('DENIS FONSECA', 6, 2010)
# ]
# colNames = ['nome', 'mes', 'ano']
# df = spark.createDataFrame(data, colNames)
# df.show(truncate=False)

In [48]:
# df\
#     .filter("mes<=6")\
#     .filter("ano=2009")\
#     .orderBy('mes', ascending=False)\
#     .show(truncate=False)

In [49]:
# df\
#     .filter(df.mes <= 6)\
#     .filter(df.ano = 2009)\
#     .show(truncate=False)

In [50]:
# df\
#     .filter((df.mes <= 6) & (df.ano == 2009))\
#     .show(truncate=False)

In [51]:
empresas\
    .select('razao_social', 'natureza_juridica', 'porte_da_empresa', 'capital_social_empresa')\
    .filter(f.upper(empresas['razao_social']).like('RESTAURANTE%'))\
    .show(15, False)

+------------------------------------------+-----------------+----------------+----------------------+
|razao_social                              |natureza_juridica|porte_da_empresa|capital_social_empresa|
+------------------------------------------+-----------------+----------------+----------------------+
|RESTAURANTE IMIGRANTE PORTUGUES LTDA.     |2062             |5               |0.0                   |
|RESTAURANTE E PETISCARIA THE-DELICIA LTDA |2062             |5               |0.0                   |
|RESTAURANTE PONTO NOBRE EIRELI            |2305             |1               |105000.0              |
|RESTAURANTE ALEGRIA ALEGRIA LTDA          |2062             |1               |5000.0                |
|RESTAURANTE D GABI EIRELI                 |2305             |1               |95400.0               |
|RESTAURANTE DONA CHICA LTDA               |2062             |1               |5000.0                |
|RESTAURANTE SABOR REGIONAL LTDA           |2062             |1          

In [52]:
# EXERCICIO

In [53]:
# data = [
#     ('CARMINA RABELO', 4, 2010),
#     ('HERONDINA PEREIRA', 6, 2009),
#     ('IRANI DOS SANTOS', 12, 2010),
#     ('JOAO BOSCO DA FONSECA', 3, 2009),
#     ('CARLITO SOUZA', 1, 2010),
#     ('WALTER DIAS', 9, 2009),
#     ('BRENO VENTUROSO', 1, 2009),
#     ('ADELINA TEIXEIRA', 5, 2009),
#     ('ELIO SILVA', 7, 2010),
#     ('DENIS FONSECA', 6, 2010)
# ]
# colNames = ['nome', 'mes', 'ano']
# df = spark.createDataFrame(data, colNames)
# df.show(truncate=False)

In [54]:
# df\
#     .filter(df.nome.like('%C%'))\
#     .show(truncate=False)

In [55]:
# df\
#     .select('nome')\
#     .like('C%')\
#     .show(truncate=False)

In [56]:
# df\
#     .filter(df.nome.like('C%'))\
#     .show(truncate=False)

In [57]:
# df\
#     .select('nome')\
#     .like('%C%')\
#     .show(truncate=False)

In [58]:
#  SUMARIZANDO OS DADOS

In [59]:
socios\
    .select(f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .where('ano_de_entrada >= 2010')\
    .groupBy('ano_de_entrada')\
    .count()\
    .orderBy('ano_de_entrada', ascending=True)\
    .show()

+--------------+------+
|ano_de_entrada| count|
+--------------+------+
|          2010| 79337|
|          2011| 83906|
|          2012| 80101|
|          2013| 83919|
|          2014| 80590|
|          2015| 80906|
|          2016| 81587|
|          2017| 90221|
|          2018| 99935|
|          2019|118248|
|          2020|125927|
|          2021| 56316|
+--------------+------+



In [60]:
empresas\
    .select('cnpj_basico', 'porte_da_empresa', 'capital_social_empresa')\
    .groupBy('porte_da_empresa')\
    .agg(
        f.avg('capital_social_empresa').alias('capital_social_medio'),
        f.count('cnpj_basico').alias('frequencia')
    )\
    .orderBy('porte_da_empresa', ascending=True)\
    .show()

+----------------+--------------------+----------+
|porte_da_empresa|capital_social_medio|frequencia|
+----------------+--------------------+----------+
|            NULL|    8.35421888053467|      5985|
|               1|  339994.53313506936|   3129043|
|               3|  2601001.7677092673|    115151|
|               5|   708660.4208249798|   1335500|
+----------------+--------------------+----------+



In [61]:
empresas\
    .select('capital_social_empresa')\
    .summary()\
    .show()
#  .summary('count', 'mean', 'stddev', 'min', '25%', '50%', '75%', 'max')

+-------+----------------------+
|summary|capital_social_empresa|
+-------+----------------------+
|  count|               4585679|
|   mean|     503694.5478542675|
| stddev|  2.1118691490537405E8|
|    min|                   0.0|
|    25%|                   0.0|
|    50%|                1000.0|
|    75%|                7000.0|
|    max|      3.22014670262E11|
+-------+----------------------+



In [62]:
# Exercicio


In [63]:
# data = [
#     ('CARLOS', 'MATEMÁTICA', 7),
#     ('IVO', 'MATEMÁTICA', 9),
#     ('MÁRCIA', 'MATEMÁTICA', 8),
#     ('LEILA', 'MATEMÁTICA', 9),
#     ('BRENO', 'MATEMÁTICA', 7),
#     ('LETÍCIA', 'MATEMÁTICA', 8),
#     ('CARLOS', 'FÍSICA', 2),
#     ('IVO', 'FÍSICA', 8),
#     ('MÁRCIA', 'FÍSICA', 10),
#     ('LEILA', 'FÍSICA', 9),
#     ('BRENO', 'FÍSICA', 1),
#     ('LETÍCIA', 'FÍSICA', 6),
#     ('CARLOS', 'QUÍMICA', 10),
#     ('IVO', 'QUÍMICA', 8),
#     ('MÁRCIA', 'QUÍMICA', 1),
#     ('LEILA', 'QUÍMICA', 10),
#     ('BRENO', 'QUÍMICA', 7),
#     ('LETÍCIA', 'QUÍMICA', 9)
# ]
# colNames = ['nome', 'materia', 'nota']
# df = spark.createDataFrame(data, colNames)
# df.show()

In [64]:
# df = df.withColumn('status', f.when(df.nota >= 7, "APROVADO").otherwise("REPROVADO"))
# df.show()

In [65]:
# data = [
#     ('CARLOS', 'MATEMÁTICA', 7),
#     ('IVO', 'MATEMÁTICA', 9),
#     ('MÁRCIA', 'MATEMÁTICA', 8),
#     ('LEILA', 'MATEMÁTICA', 9),
#     ('BRENO', 'MATEMÁTICA', 7),
#     ('LETÍCIA', 'MATEMÁTICA', 8),
#     ('CARLOS', 'FÍSICA', 2),
#     ('IVO', 'FÍSICA', 8),
#     ('MÁRCIA', 'FÍSICA', 10),
#     ('LEILA', 'FÍSICA', 9),
#     ('BRENO', 'FÍSICA', 1),
#     ('LETÍCIA', 'FÍSICA', 6),
#     ('CARLOS', 'QUÍMICA', 10),
#     ('IVO', 'QUÍMICA', 8),
#     ('MÁRCIA', 'QUÍMICA', 1),
#     ('LEILA', 'QUÍMICA', 10),
#     ('BRENO', 'QUÍMICA', 7),
#     ('LETÍCIA', 'QUÍMICA', 9)
# ]
# colNames = ['nome', 'materia', 'nota']
# df = spark.createDataFrame(data, colNames)
# df = df.withColumn('status', f.when(df.nota >= 7, "APROVADO").otherwise("REPROVADO"))
# df.show()

In [66]:
# df\
#     .select('nota')\
#     .agg('min', '25%', '50%', '75%', 'max')\
#     .show()

In [67]:
# df\
#     .count()\
#     .orderBy('status', ascending=True)\
#     .groupBy('status')\
#     .show()

In [68]:
# df\
#     .select('nota')\
#     .summary()\
#     .show()

In [69]:
# df\
#     .groupBy('status')\
#     .count()\
#     .orderBy('status', ascending=True)\
#     .show()

In [70]:
# df\
#     .select('nota')\
#     .summary('min', '25%', '50%', '75%', 'max')\
#     .show()

In [71]:
# df\
#     .groupBy('status')\
#     .count()\
#     .orderBy('status', ascending=True)\
#     .show()

In [72]:
# Juntando DataFrames

In [73]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificação_resposavel: integer (nullable = true)
 |-- capital_social_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente-federativo_responsavel: string (nullable = true)



In [74]:
socios.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: date (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [75]:
estabelecimentos.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: string (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: string (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- data_de_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: string (null

In [76]:
empresas_join = estabelecimentos.join(empresas, 'cnpj_basico', how='inner')

In [77]:
empresas_join.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: string (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: string (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- data_de_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: string (null

In [78]:
freq = empresas_join\
    .select(
        'cnpj_basico',
        f.year('data_de_inicio_atividade').alias('data_de_inicio'))\
    .where('data_de_inicio >= 2010')\
    .groupBy('data_de_inicio')\
    .agg(f.count('cnpj_basico').alias('frequencia'))\
    .orderBy('data_de_inicio', ascending=True)

In [79]:
freq.show(5)

+--------------+----------+
|data_de_inicio|frequencia|
+--------------+----------+
|          2010|    185176|
|          2011|    206689|
|          2012|    279134|
|          2013|    238141|
|          2014|    242778|
+--------------+----------+
only showing top 5 rows



In [80]:
freq.union(
    freq.select(
        f.lit('Total').alias('data_de_inicio'),
        f.sum('frequencia').alias('frequencia')
    )
).show()

+--------------+----------+
|data_de_inicio|frequencia|
+--------------+----------+
|          2010|    185176|
|          2011|    206689|
|          2012|    279134|
|          2013|    238141|
|          2014|    242778|
|          2015|    255233|
|          2016|    319193|
|          2017|    284579|
|          2018|    330380|
|          2019|    391154|
|          2020|    480538|
|          2021|    183905|
|         Total|   3396900|
+--------------+----------+



In [81]:
idades = spark.createDataFrame(
    [
        ('CARLOS', 15),
        ('IVO', 14),
        ('MÁRCIA', 16),
        ('LEILA', 17),
        ('LETÍCIA', 14)
    ],
    ['nomes', 'idades']
)

notas = spark.createDataFrame(
    [
        ('CARLOS', 10),
        ('MÁRCIA', 1),
        ('LEILA', 10),
        ('BRENO', 7),
        ('LETÍCIA', 9)
    ],
    ['nomes', 'notas']
)

In [82]:
idades.join(notas, 'nomes', how='outer')\
    .sort('nomes')\
    .show()

+-------+------+-----+
|  nomes|idades|notas|
+-------+------+-----+
|  BRENO|  NULL|    7|
| CARLOS|    15|   10|
|    IVO|    14| NULL|
|  LEILA|    17|   10|
|LETÍCIA|    14|    9|
| MÁRCIA|    16|    1|
+-------+------+-----+



In [83]:
empresas.createOrReplaceTempView("empresasView")

In [84]:
spark.sql("SELECT * FROM empresasView").show(5)

+-----------+--------------------+-----------------+-----------------------+----------------------+----------------+---------------------------+
|cnpj_basico|        razao_social|natureza_juridica|qualificação_resposavel|capital_social_empresa|porte_da_empresa|ente-federativo_responsavel|
+-----------+--------------------+-----------------+-----------------------+----------------------+----------------+---------------------------+
|        306|FRANCAMAR REFRIGE...|             2240|                     49|                   0.0|               1|                       NULL|
|       1355|BRASILEIRO & OLIV...|             2062|                     49|                   0.0|               5|                       NULL|
|       4820|REGISTRO DE IMOVE...|             3034|                     32|                   0.0|               5|                       NULL|
|       5347|ROSELY APARECIDA ...|             2135|                     50|                   0.0|               5|              

In [85]:
spark\
  .sql("""
    SELECT * FROM empresasView
    WHERE capital_social_empresa = 50
  """)\
  .show(5)

+-----------+--------------------+-----------------+-----------------------+----------------------+----------------+---------------------------+
|cnpj_basico|        razao_social|natureza_juridica|qualificação_resposavel|capital_social_empresa|porte_da_empresa|ente-federativo_responsavel|
+-----------+--------------------+-----------------+-----------------------+----------------------+----------------+---------------------------+
|   17350147|ERIK MARCELO DOS ...|             2135|                     50|                  50.0|               1|                       NULL|
|   17833214|ALEXANDRE MACHADO...|             2135|                     50|                  50.0|               1|                       NULL|
|   20860830|YASMIN MOURA DA F...|             2135|                     50|                  50.0|               1|                       NULL|
|   22242856|JOAO CESAR MESSIA...|             2135|                     50|                  50.0|               1|              

In [86]:
spark\
  .sql("""
      SELECT porte_da_empresa, MEAN(capital_social_empresa) AS media
      FROM empresasView
      GROUP BY porte_da_empresa
  """)\
  .show()

+----------------+------------------+
|porte_da_empresa|             media|
+----------------+------------------+
|            NULL|  8.35421888053467|
|               1|339994.53313506936|
|               3|2601001.7677092673|
|               5| 708660.4208249798|
+----------------+------------------+



In [87]:
empresas_join.createOrReplaceTempView("empresasView")

In [88]:
freq = spark\
  .sql("""
      SELECT YEAR(data_de_inicio_atividade) AS data_de_inicio, COUNT(cnpj_basico) AS frequencia
      FROM empresasView
      WHERE YEAR(data_de_inicio_atividade) >= 2010
      GROUP BY data_de_inicio
      ORDER BY data_de_inicio
  """)

freq\
    .show()

+--------------+----------+
|data_de_inicio|frequencia|
+--------------+----------+
|          2010|    185176|
|          2011|    206689|
|          2012|    279134|
|          2013|    238141|
|          2014|    242778|
|          2015|    255233|
|          2016|    319193|
|          2017|    284579|
|          2018|    330380|
|          2019|    391154|
|          2020|    480538|
|          2021|    183905|
+--------------+----------+



In [89]:
freq.createOrReplaceTempView("freqView")

In [91]:
# spark\
#   .sql("""
#       SELECT *
#           FROM freqView
#       UNION ALL
#       SELECT 'Total' AS data_de_inicio, SUM(count) AS count
#           FROM freqView
#   """)\
#   .show()


spark\
  .sql("""
      SELECT *
          FROM freqView
      UNION ALL
      SELECT 'Total' AS data_de_inicio, SUM(frequencia) AS count  -- Change 'count' to 'frequencia'
          FROM freqView
  """)\
  .show()



+--------------+----------+
|data_de_inicio|frequencia|
+--------------+----------+
|          2010|    185176|
|          2011|    206689|
|          2012|    279134|
|          2013|    238141|
|          2014|    242778|
|          2015|    255233|
|          2016|    319193|
|          2017|    284579|
|          2018|    330380|
|          2019|    391154|
|          2020|    480538|
|          2021|    183905|
|         Total|   3396900|
+--------------+----------+

